In [35]:
!pip install mtcnn
from mtcnn.mtcnn import MTCNN
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import cv2
from numpy import load
from numpy import expand_dims
from numpy import asarray
from numpy import savez_compressed
from keras.models import load_model
import os
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
import pickle


def extract_face(file,required_size=(160,160)):
    image = Image.open(file)
    print('Working on image:',file)
    image = image.convert('RGB')
    pixels = np.asarray(image)
    detector = MTCNN()
    results = detector.detect_faces(pixels)
    try:
        x,y,w,h = results[0]['box']
    except:
        print('face not found:')
    x,y = abs(x),abs(y)
    face = pixels[y:y+h,x:x+w]
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = np.asarray(image)
    return face_array




Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [36]:
# get the face embedding for one face
def get_embedding(model, face_pixels):
	# scale pixel values
	face_pixels = face_pixels.astype('float32')
	print(face_pixels.shape)
	# standardize pixel values across channels (global)
	mean, std = face_pixels.mean(), face_pixels.std()
	face_pixels = (face_pixels - mean) / std
	# transform face into one sample
	samples = expand_dims(face_pixels, axis=0)
	# make prediction to get embedding
	yhat = model.predict(samples)
	return yhat[0]



In [40]:
#Classes
people = ['Ben Affleck','Cherishma Shankar', 'Elton John', 'Jerry Seinfeld', 'Madonna', 'Mindy Kaling']

#load FaceNet model from keras
keras_model = load_model('facenet_keras.h5')

#Normalizer
encoder = Normalizer(norm='l2')

#SVM model
model = pickle.load(open('svm_model.sav','rb'))

face_pixels = extract_face('test_image5.jpg')

#Get word embedding for face
face_embedding = get_embedding(keras_model,face_pixels)

#normalize the embedding
face_embedding = encoder.transform(face_embedding.reshape(1,-1))


#predict from SVM model
predicted_label = model.predict(face_embedding)
print(predicted_label)
predicted_name  = people[predicted_label[0]]
print(predicted_name)




Working on image: test_image5.jpg


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator SVC from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


(160, 160, 3)
[1]
Cherishma Shankar
